<a href="https://colab.research.google.com/github/sayarghoshroy/Summaformers/blob/main/laySumm_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import spacy
from nltk import word_tokenize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

In [ ]:
import pickle
import nltk
import numpy as np
import os
from nltk.tokenize import sent_tokenize

import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import pickle as pkl

!pip install transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import GPT2Model, GPT2Config, GPT2Tokenizer, GPT2Tokenizer, GPT2LMHeadModel
from transformers import BartTokenizer, BartForConditionalGeneration

from tqdm import tqdm

filename = open("drive/My Drive/Scientific_Doc_Summarization_Raw_Data/Pickled Datasets/laySumm_Test.pkl", "rb")
store = pkl.load(filename)
print("Number of Datapoints:", len(store))

# Experiment with generated extractive summaries
# filename = open("drive/My Drive/Scientific_Doc_Summarization_Raw_Data/summa_ext_res.pkl", "rb")
# store_ext = pkl.load(filename)
# print("Number of Datapoints:", len(store_ext))

def get_text(units):
    text = ""

    # To consider a set of paragraphs
    count_pg = 0
    pg_id = 0

    for sent in units:
        if sent == 'PARAGRAPH':
            count_pg += 1
    
    for sent in units:
        if sent == 'PARAGRAPH':
            pg_id += 1
            continue
        
        # Tried out sectional input variations
        # if pg_id != 1 and pg_id != 2 and pg_id != count_pg:
        #   continue
        
        tokens = nltk.word_tokenize(sent)
        
        for token in tokens:
            text += token.strip().rstrip("\n") + " "
            
    return text

def get_tops(units):
    text = ""
    count = 0
    word_count = 0
    
    for sent in units:
        if sent == 'PARAGRAPH':
            continue

        text += sent + "\n\n"
        count += 1
        word_count += len(sent.split(" "))
        if word_count > 150:
        	break
    
    return text

count = 0

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
our_dict = {'abstract': {'url': 'ourpaper', 'id': '1', 'title':'Scientific Document Summarization @ LaySumm 20 and LongSumm 20', 'abstract': ['PARAGRAPH',
   "Automatic text summarization has been widely studied as an important task in natural language processing.",
 "Traditionally, various feature engineering and machine learning based systems have been proposed for extractive as well as abstractive text summarization.",
 "Recently, deep learning based, specifically Transformer-based systems have been immensely popular.",
"Summarization is a cognitively challenging task - extracting summary worthy sentences is laborious, and expressing semantics in brief when doing abstractive summarization is complicated.",
"In this paper, we specifically look at the problem of summarizing scientific research papers from multiple domains.",
"We differentiate between two types of summaries, namely, (a) LaySumm: A very short summary that captures the essence of the research paper in layman terms restricting overtly specific technical jargon and (b) LongSumm: A much longer detailed summary aimed at providing specific insights into various ideas touched upon in the paper.",
"While leveraging latest Transformer-based models, our systems are simple, intuitive and based on how specific paper sections contribute to human summaries of the two types described above."]},
'fulltext': {'url': 'ourpaper', 'id': '1', 'title':'Scientific Document Summarization @ LaySumm 20 and LongSumm 20', 'introduction': '...', 'conclusion': '...', 'methods': '...'}}

In [ ]:
# our_dict stores the dictionary for our own paper
# To generate LaySumm for our paper:
# store = [our_dict]

In [ ]:
def cleanup(summary, abstract):
    abs_tokens = {}
    
    for x in word_tokenize(abstract):
        abs_tokens[x] = 0
    
    tokens = []
    
    for x in word_tokenize(summary):
      if not x.isalpha() and x not in abs_tokens and x != '.':
        pass
        # print(x)
      else:
        tokens.append(x)
            
    fixed_string = ""
    for token in tokens:
        if token != '.':
            fixed_string += (token + " ")
            
        else:
            fixed_string += ".\n\n"
    
    return fixed_string

In [ ]:
# For saved BART models
model_statedict_path = "drive/My Drive/Scientific_Doc_Summarization_Raw_Data/Saved Models/bart_abs_only_best.pt"
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn').to(device)
model.load_state_dict(torch.load(model_statedict_path, map_location=torch.device('cpu')))

# For saved T5 models
# model_statedict_path = "./drive/My Drive/Scientific_Doc_Summarization_Raw_Data/t5.pt"
# tokenizer = T5Tokenizer.from_pretrained('t5-base')
# model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)
# model.load_state_dict(torch.load(model_statedict_path))

<All keys matched successfully>

In [ ]:
# A directory with the provided model name will be created which will hold the submission files

model_name = 'model'

if os.path.isdir(model_name):
    os.rmdir(model_name)
os.mkdir(model_name)

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
def check(list_str, term):
  for stri in list_str:
    if term in stri:
      return (True, stri)
  return (False, "")

count = 0
for paper in store:
    saved = check(paper['fulltext'].keys(), 'result')
    if saved[0]:
      doc_results = get_text(paper['fulltext'][saved[1]]).replace("\n"," ")
    else:
      doc_results = ""

In [ ]:
missing = 0
for paper in store:
  if 'title' in paper['fulltext']:
    head = paper['fulltext']['title']
  else:
    missing += 1

In [ ]:
all_summaries = {}

for paper in tqdm(store):
    no_conc = 0
    count += 1

    abstract = get_text(paper['abstract']['abstract'])

    # abstract = store_ext[paper['abstract']['id']].replace("\n\n", "")
    # to use the generated extractive summaries as input
    # print(abstract)

    if 'title' in paper['fulltext']:
      head = paper['fulltext']['title']
    else:
      head = ""
    
    if 'discussion' in paper['fulltext'].keys():
      doc_diss = get_text(paper['fulltext']['discussion'])
    else:
      doc_diss = ""

    if 'conclusion' in paper['fulltext'].keys():
      doc_conc = get_text(paper['fulltext']['conclusion']).replace("\n"," ")
    
    elif 'conclusions' in paper['fulltext'].keys():
      doc_conc = get_text(paper['fulltext']['conclusions']).replace("\n"," ")
    
    else:
      no_conc += 1
      doc_conc = ""

    if 'introduction' in paper['fulltext'].keys():
      doc_intro = get_text(paper['fulltext']['introduction']).replace("\n"," ")

    else:
      doc_intro = ""

    saved = check(paper['fulltext'].keys(), 'result')
    if saved[0]:
      doc_results = get_text(paper['fulltext'][saved[1]]).replace("\n"," ")
    else:
      dec_results = ""

    for key in paper['fulltext'].keys():
    	# retrieves sections one by one
        if key == 'title':
            continue
        
        section = get_text(paper['fulltext'][key])

    # use saved models to generate model output here
    # section-wise summarizers on selected sections
    # use-budget concat code
    # use the abstractor model
    # separate into sentences with blank line in between, refer: get_top_3()
    
    # source = tokenizer.batch_encode_plus([abstract + " " + doc_conc], max_length= 512, pad_to_max_length=True,return_tensors='pt',truncation=True)
    # source = tokenizer.batch_encode_plus(["summarize: " + abstract + " " + doc_conc], max_length= 512, pad_to_max_length=True,return_tensors='pt',truncation=True)

    source = tokenizer.batch_encode_plus([abstract], max_length= 512, pad_to_max_length=True,return_tensors='pt',truncation=True)

    source_ids = source['input_ids'].reshape(1,-1).to(device, dtype = torch.long)
    source_mask = source['attention_mask'].reshape(1,-1).to(device, dtype = torch.long)
    
    generated_ids = model.generate(
            input_ids = source_ids,
            attention_mask = source_mask, 
            min_len = 120,
            max_length = 140,
            num_beams=2,
            repetition_penalty=1.8, 
            length_penalty=1.0, 
            early_stopping=True
            )
    
    model_laySumm = cleanup([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids][0], abstract).replace("( ", "(").replace(" )", ")").replace("( ", "(").replace(" )", ")").replace("( ", "(").replace(" )", ")").replace(" .", ".").replace(" ,", ",")
    # model_laySumm = '.\n\n'.join(model_laySumm.split('.'))
    # model_laySumm = model_laySumm.replace(')', ' )')
    
    output = "https://doi.org/" + paper['abstract']['url']
    output += "\n\nLAYSUMM\n\nTITLE\n\n" + paper['abstract']['title']
    output += "\n\nPARAGRAPH\n\n"
    output += model_laySumm
    output += "\n"

    all_summaries[paper['abstract']['id']] = model_laySumm

    file_out = open("./" + model_name + "/" + paper['abstract']['id'] + "_LAYSUMM.TXT", "w+")
    file_out.write(output)
    file_out.close()

In [ ]:
!zip -r ./model.zip ./model/

In [ ]:
# filename = open("bart_best.pkl", "wb")
# pickle.dump(all_summaries, filename)

In [ ]:
# Trying out a paraphraser

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
tokenizer_p = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model_p = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws").to("cuda")

sentence =  'The presence of a single large portosystemic shunts ( SPSS ) is associated with complications, especially overt hepatic encephalopathy ( oHE ) in patients with liver cirrhosis. In this study, we show that a total cross-sectional SPST area of 83 mm2 or more increases the risk for oHE and mortality in patients who develop SPSs > 1. Therefore, screening for a large S PSS should be considered when stratifying patients with cirrhotic liver disease.'

text =  "paraphrase: " + sentence

encoding = tokenizer_p.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
outputs = model_p.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    top_k=120,
    top_p=0.9,
    max_length=150,
    min_length=125,
    early_stopping=False,
    num_return_sequences=1)

for output in outputs:
    line = tokenizer_p.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(line)

# Results were not that great

In [ ]:
# ^_^ Thank You